GOAL:

Clustering: LDA and NMG for Topic Modeling
Classification: Random Forest, Continuous Data - Classifiy Elite Status -- Find people who are qualified to be elite but are not. 
Regression: 



In [102]:
%pylab inline
import pandas as pd
import numpy as np
import seaborn as sns
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition.online_lda import LatentDirichletAllocation
import string
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

Populating the interactive namespace from numpy and matplotlib


In [2]:
user = pd.read_pickle('data/user')
business =pd.read_pickle('data/business')
review = pd.read_pickle('data/review')
# top_restaurant = pd.read_csv('updated/high_reviews_restaurant.csv')

In [214]:
review.columns

Index(['user_id', 'review_id', 'text', 'votes_cool', 'business_id',
       'votes_funny', 'stars', 'date', 'votes_useful'],
      dtype='object')

In [183]:
user.columns

Index(['index', 'yelping_since', 'compliments_plain', 'review_count',
       'compliments_cute', 'compliments_writer', 'fans', 'compliments_note',
       'compliments_hot', 'compliments_cool', 'compliments_profile',
       'average_stars', 'compliments_more', 'elite', 'name', 'user_id',
       'votes_cool', 'compliments_list', 'votes_funny', 'compliments_photos',
       'compliments_funny', 'votes_useful', 'elite_year', 'elite_since',
       'year_before_elite'],
      dtype='object')

In [215]:
# business.city.value_counts()

In [216]:
# business.city.unique()

In [217]:
# business.columns

In [184]:
elite_user = (user['user_id'].ix[user['elite_since'] != 0])
not_elite_user = (user['user_id'].ix[user['elite_since'] == 0])

In [185]:
elite_index = review['user_id'].isin(elite_user )
not_elite_index = review['user_id'].isin(not_elite_user)

In [186]:
elite_review = review[elite_index ]
not_elite_review = review[not_elite_index]

In [187]:
elite_review.head(2)

,user_id,review_id,text,votes_cool,business_id,votes_funny,stars,date,votes_useful
25,tAKjY3bQXH51msJbOHYPmQ,uf61rPucuICXhSPXlZ1hIQ,"After a morning of Thrift Store hunting, a fri...",2,mVHrayjG3uZ_RLHkLj-AMg,0,5,2014-11-03,3
27,a0ULPTNi05w-BNVovVUz2w,1W1vq7D1ofwFKbCA_GWu_A,"From reading the other reviews, I see this pla...",4,mVHrayjG3uZ_RLHkLj-AMg,1,5,2015-03-06,1


In [191]:
not_elite_review.head(1)

,user_id,review_id,text,votes_cool,business_id,votes_funny,stars,date,votes_useful
0,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",0,5UmKMjUEUNdYWqANhGckJw,0,4,2012-08-01,0


In [193]:
e_all,good,bad = get_user_reviews('tAKjY3bQXH51msJbOHYPmQ')

In [192]:
ne_all,g,b = get_user_reviews('PUFPaY9KxDAcGqfsorJp3Q')

In [199]:
vectorizer(ne_all)

['yes',
 'expecting',
 'fry',
 'frozen',
 'feel',
 'fashioned',
 'family',
 'experience',
 'expensive',
 'excellent',
 'going',
 'enjoy',
 'eat',
 'drink',
 'doughy',
 'dish',
 'dinnertime',
 'dining',
 'garden',
 'got',
 'large',
 'india',
 'kennywood',
 'ive',
 'institution',
 'instead',
 'ingredient',
 'inexpensive',
 'indian',
 'hungarian',
 'great',
 'hot',
 'homemade',
 'home',
 'hoagie',
 'hated',
 'hate',
 'grilled',
 'different',
 'desired',
 'depending',
 'area',
 'best',
 'batch',
 'baked',
 'bacon',
 'ave',
 'authentic',
 'atmosphere',
 'alternative']

In [198]:
vectorizer(e_all)

['misaki',
 'jokingly',
 'joined',
 'joes',
 'joemerican',
 'jive',
 'jimmy',
 'jet',
 'jerky',
 'jelly',
 'jasmine',
 'james',
 'jacket',
 'jacked',
 'jack',
 'befitting',
 'itswellfood',
 'itso',
 'itsa',
 'itand',
 'italy',
 'iswelldeli',
 'iswelladorable',
 'joke',
 'jonesing',
 'belching',
 'josh',
 'kidney',
 'kheer',
 'kfc',
 'keyword',
 'kevin',
 'kentucky',
 'kelly',
 'keg',
 'keen',
 'kaya',
 'karaokeit',
 'kapoor',
 'kale',
 'kaiser',
 'justhealth',
 'jury',
 'jump',
 'jummyjummy',
 'jukebox',
 'juiceless',
 'judging',
 'issuebecause',
 'isdo']

In [234]:
test_index =(review['business_id'] == 'SsGNAc9U-aKPZccnaDtFkA')  

In [235]:
test = review[test_index]

In [236]:
test.head(1)

,user_id,review_id,text,votes_cool,business_id,votes_funny,stars,date,votes_useful
1795637,nEYPahVwXGD2Pjvgkm7QqQ,TZIDxRA_49jy3wxuQ_ZUjw,When people asked me what I had at Meat and Po...,5,SsGNAc9U-aKPZccnaDtFkA,2,4,2011-06-01,6


In [238]:
test.set_index('business_id')['user_id'].to_dict()

{'SsGNAc9U-aKPZccnaDtFkA': 'dL6PnP11WKAafsL26-22aA'}

In [232]:
sum(review['business_id'] == 'SsGNAc9U-aKPZccnaDtFkA') 

1096

In [233]:
review['user_id'].value_counts()[:10]

9A2-wSoBUxlMd3LwmlGrrQ    937
3gIfcQq5KxAegwCPXc83cQ    813
Iu3Jo9ROp2IWC9FwtWOaUQ    729
pEVf8GRshP9HUkSpizc9LA    689
ia1nTRAQEaFWv0cwADeK7g    682
kGgAARL2UmvCcTRfiscjug    665
uZbTb-u-GVjTa2gtQfry5g    650
glRXVWWD6x1EZKfjJawTOg    611
5lq4LkrviYgQ4LJNsBYHcA    569
p7dvahRPZIWL7T6pFA_0dg    560
Name: user_id, dtype: int64

In [116]:
def clean_text(x):
    wordnet = WordNetLemmatizer()
    words =  [re.sub('['+string.punctuation+']', '', i.lower().replace('\n','')) for i in x.split()]
    lemmetized_words = [wordnet.lemmatize(word) for word in words ]
    return " ".join(lemmetized_words)

In [170]:
# vectorizer = CountVectorizer(max_features=1000,max_df=0.95,  min_df=2,stop_words='english')
# matrix = vectorizer.fit_transform(good) # documents x terms 

In [174]:
freqs = [(word, matrix.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
print (sorted(freqs, key = lambda x: -x[1])[:20])


[('good', 339), ('menu', 294), ('sauce', 255), ('fresh', 254), ('service', 245), ('great', 232), ('dish', 228), ('like', 225), ('food', 216), ('place', 194), ('chicken', 190), ('order', 184), ('just', 184), ('salad', 177), ('cheese', 174), ('really', 157), ('portion', 154), ('dishes', 152), ('rice', 137), ('flavor', 133)]


In [181]:
matrix = vectorizer.fit_transform(bad2) # documents x terms 

In [182]:
freqs = [(word, matrix.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
print (sorted(freqs, key = lambda x: -x[1])[:20])


[('good', 798), ('like', 543), ('got', 536), ('food', 529), ('just', 493), ('place', 489), ('really', 388), ('went', 323), ('chicken', 251), ('bf', 250), ('came', 244), ('wasn', 218), ('didn', 217), ('pretty', 213), ('restaurant', 212), ('kind', 195), ('time', 193), ('little', 192), ('ordered', 191), ('think', 188)]


In [188]:
# indices = np.argsort(vectorizer.idf_)[::-1]
# features = vectorizer.get_feature_names()
# top_n = 10
# top_features = [features[i] for i in indices[:top_n]]

In [ ]:
lectures = ["this is some food", "this is some drink"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(lectures)
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
print top_features

In [197]:
def vectorizer(X):
    x = []
    for doc in X:
        x.append(clean_text(doc))
    tfidf = TfidfVectorizer(stop_words='english')
    x = tfidf.fit_transform(x)
    indices = np.argsort(tfidf.idf_)[::-1]
    features = tfidf.get_feature_names()
    top_features = [features[i] for i in indices[:50]]
    return top_features

In [189]:
# vectorizer(good2)

In [126]:
# review['word'] = [clean_text(x) for x in review['text']]

In [ ]:
# Counter(review['business_id'])
# review_count = Counter(review['business_id'])

In [ ]:
review_count

In [ ]:
print(1)

In [224]:
review_count = Counter(review['business_id']) # number of unique restaurants
print(len(review_count))
top_restaurant = [(k,v)for k,v in review_count.items() if v > 300]#number of restaurants that have more than 300 reviews
sorted_by_count = sorted(top_restaurant, key=lambda tup: tup[1])
sorted_by_count[-300:]

26629


[('u6EUXOSFnjxvzLll4D21bA', 564),
 ('H3SaoIWnlG-HvGQ8v6J6yg', 567),
 ('MhMFi7hae_KsaK8lHcg0iQ', 567),
 ('aYAlzKHwXQn6JNLweRnZjQ', 569),
 ('QcTj9AgGJA0ifMHCCPNblg', 571),
 ('v1aLzjTphPc6IGIy9k96vg', 572),
 ('cp39VURFby4A6u9m2It8jg', 573),
 ('6IRoWd6N__OVMNQfvu1Ccw', 573),
 ('EHIZFzQQ_mjzwyjcaToPtQ', 575),
 ('cN6aBxe2mQvrQlzk26LyRQ', 577),
 ('UpMR63WZgIn8VDKnYpDwXA', 578),
 ('84bLhe1aibxNmogtqSO4rA', 579),
 ('Lq6CDJ2yHmMs4N7jscQhHQ', 579),
 ('63d0v1xVzzahrY7zLoy_FA', 580),
 ('6imLt53br7SJ3av07jjH7w', 581),
 ('m9haUi2_1lHR2eAKBhKwDg', 582),
 ('23SJEjfyU5gxY2chAKhgcw', 583),
 ('h_WtRK_umSU8LCCmAeqbFA', 583),
 ('itFwrNtjrQL5NPMYTeD44g', 584),
 ('cBpJIOrVXotDI0XAZH_k0g', 585),
 ('bgTB6MgdVQssXhkNJ7qIfw', 586),
 ('guLkdetJ603Bw_A6tuecIg', 586),
 ('N8cJk5s9yW3fLeJyHQeRAA', 586),
 ('b5cEoKR8iQliq-yT2_O0LQ', 587),
 ('_zrSQdISpB_qry9tb098rw', 587),
 ('uEJQSIjWui-TDWXaGlcqyQ', 588),
 ('mpDxBBGywUE6GRRKja3sBA', 590),
 ('-5RN56jH78MV2oquLV_G8g', 591),
 ('WzDvQdS7MQrWK7vyDjdR4Q', 593),
 ('5FLRsYrT6w2

In [190]:
def get_user_reviews(user_id):
    review_index = review['user_id'] == user_id
    user_review = review[review_index]
    good = user_review.ix[user_review['stars'] > 3.5]
    bad = user_review.ix[user_review['stars'] <= 3.5]
    return user_review.text,good.text, bad.text

In [29]:
good2, bad2 = get_user_reviews('3gIfcQq5KxAegwCPXc83cQ')

In [212]:
a,good, bad = get_user_reviews('9A2-wSoBUxlMd3LwmlGrrQ')

In [35]:
tfidf = TfidfVectorizer(stop_words='english')
x = tfidf.fit_transform(good)

In [39]:
# lectures = ["this is some food", "this is some drink"]
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(lectures)
indices = np.argsort(tfidf.idf_)[::-1]
features = tfidf.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
print (top_features)


['ziplock', 'indicated', 'insisting', 'insanity', 'insanely', 'ins', 'inner', 'inlaid', 'inherit', 'inhale', 'ingested', 'ingenious', 'inflating', 'inevitable', 'induce', 'incurable', 'inspirada', 'increased', 'increase', 'inconveniently']


In [37]:
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
print top_features

<2x6201 sparse matrix of type '<class 'numpy.float64'>'
	with 187 stored elements in Compressed Sparse Row format>

In [213]:
lda_tm(good)

Topic 1: beesting, piroshky, worry, contains, 11pm, loose, ritzcracker, bros, returning, saw, lent, bare, overwhelm, szechuan, multipage, billion, wa, accessibly, boucher, grit
Topic 2: wa, transformation, floor, miscommunication, onion, carrot, variation, hole, ginger, dish, ny, assume, huli, hut, mexican, looking, wild, snapper, living, styrofoam
Topic 3: wa, taco, roll, place, shrimp, youre, avocado, enchilada, definitely, sauce, fresh, menu, extra, chicken, sound, sandwich, center, order, bar, friendly
Topic 4: wa, roll, sushi, vin, bottle, graham, special, included, having, delicious, pricing, sauce, lack, mahi, taxtip, wcream, lemony, number, fregola, higher
Topic 5: wa, fresh, gyro, served, piece, opening, unrestrained, dont, sauce, pommes, brown, friendly, dinein, succulent, feta, drowning, service, award, brightened, salad
Topic 6: gallo, wa, loignon, loved, whine, fast, cloth, description, weekend, pita, tanginess, ebi, overeasy, spicyzesty, rice, noodle, hate, doubt, texan, 

In [209]:
lda_tm(good2)

Topic 1: wa, closed, décor, reminiscent, pushing, tagine, sort, fixe, good, begin, gravy, general, matador, vaughns, brewed, le, secretly, unlike, significantly, pouring
Topic 2: meh, chees, blessing, placei, barely, currynothing, arcadia, watch, contraption, east, bagel, remedied, environment, thigh, stuffedosity, yellow, jack, mebut, salsa, boop
Topic 3: devil, bread, junk, salty, pastor, heel, relish, compound, ball, pricy, bean, terse, shady, whipped, burger, subbed, pack, sonoran, noticed, seven
Topic 4: wa, cocktail, woo, drink, brunch, mac, priced, cheese, stayed, pricier, 13, rich, chilaquiles, fig, delicious, solid, havent, badly, topping, pork
Topic 5: wa, good, half, pizza, lunch, creamy, item, sweet, tasty, went, nice, complaint, frustrating, make, beer, salad, tangyeverything, different, complain, ling
Topic 6: wa, good, cocktail, small, delicious, woo, drink, overall, salad, friend, place, sweet, got, like, food, mac, plate, tasty, flavor, priced
Topic 7: sampler, pale, t

In [208]:
lda_tm(bad)

Topic 1: timely, increasing, necessarily, heading, important, tastier, gain, old, viewed, called, ethnic, gal, vegetarian, glued, difference, tonic, word, clothes, making, thinking
Topic 2: chop, sandwich, bashing, hot, chew, checking, feeling, dollar, stuck, hola, 62, matching, refilled, cheese, admit, calling, fishing, frazzled, apple, brisket
Topic 3: bierwurst, downright, cheek, smallsurfaced, lexusand, tall, mushy, selection, like, christmas, limitedyouve, baba, range, tow, degree, pew, sushi, kale, armands, lower
Topic 4: kfc, bent, feel, hat, hoagie, cured, middleclass, patty, lardo, tenbuck, opened, rabbit, hotel, foodnot, truly, dreck, basic, grapenut, ive, starting
Topic 5: ive, sliced, collection, wa, spud, forced, canned, anymore, haus, offering, starbucks, crew, checking, doing, joes, left, wurstlplatte, away, pair, polenta
Topic 6: oddalmost, spicy, timely, pass, vinegar, appeared, buttered, warm, unitbut, city, based, philly, net, undead, hola, reuben, forth, suggestion,

In [4]:
def categorize_reviews(business_id):
    business_index = restaurant_review['business_id'] == business_id # get the business index
    one_restaurant = restaurant_review[business_index]
    good = one_restaurant.ix[one_restaurant['stars'] > 3.5]
    bad = one_restaurant.ix[one_restaurant['stars'] <= 3.5]
    good = good.reset_index()
    bad = bad.reset_index()
    return good.text,bad.text

In [207]:
def lda_tm(X):
    x = [clean_text(x) for x in X]
    tfidf = TfidfVectorizer(stop_words='english')
    x = tfidf.fit_transform(x)
    lda = LatentDirichletAllocation(n_topics=15,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.)
    terms = tfidf.get_feature_names()
    lda.fit(x)
    C = lda.components_
    for topic,term in enumerate(C):
        top_indices = np.argsort(C)[topic][::-1][:20]
        term_ranking = [terms[i] for i in top_indices]
        print("Topic {}: {}".format(topic+1, ", ".join(term_ranking)))
    

In [11]:
def nmf_tm(x):
    tfidf = TfidfVectorizer(stop_words='english')
    x = tfidf.fit_transform(x)
    nmf = NMF(init="nndsvd",
            n_components=15,
            max_iter=200)
    terms = tfidf.get_feature_names()
    nmf.fit(x)
    W = nmf.fit_transform(x)
    H = nmf.components_
    for topic,term in enumerate(H):
        top_indices = np.argsort(H)[topic][::-1][:10]
        term_ranking = [terms[i] for i in top_indices]
        print("Topic {}: {}".format(topic+1, ", ".join(term_ranking)))
    

In [12]:
len(review)

1630712

In [8]:
top_restaurant.head(2)

,Unnamed: 0,business_id,review_count
0,0,FY_OXEVcewAQFZuu6fynLA,301
1,1,7WGBi29_vj_7RcPVxpLNHA,301


In [9]:
rating_matrix = np.zeros( shape )

TypeError: expected sequence object with len >= 0 or a single integer

In [ ]:
self.SVD_matrix = np.zeros(shape = (944, 1683), dtype = np.float32)
def modifyRatingsDataset(self): 
        ''' Assign rating values to a matrix (userid, movieid ) '''
        for index, row in self.ratings.iterrows():
            self.SVD_matrix[ int(row[0]) , int(row[1])] = float(row[2])
        return csc_matrix(self.SVD_matrix, dtype=np.float32),self.SVD_matrix